In [45]:
import pandas as pd

In [149]:
df=pd.read_csv('data/finalTrain.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [150]:
df.drop(columns=['Order_Date','ID','Restaurant_latitude','Restaurant_longitude'],axis=1,inplace=True)

In [151]:
def str_min(x):
    min=0
    if type(x)==list:
        if len(x)>1:
            min = x[1]
        else:
            min = int(float(x[0])*60)
    return int(min)

In [152]:
df['Time_Orderd_min'] = df['Time_Orderd'].fillna(df['Time_Orderd'].mode()[0]).str.split(':').apply(str_min)
df['Time_Order_picked_min'] = df['Time_Order_picked'].fillna(df['Time_Orderd'].mode()[0]).str.split(':').apply(str_min)
df.drop(columns=['Time_Orderd','Time_Order_picked'],axis=1,inplace=True)

In [153]:
#Calculating time difference order and pickup and removing the minutes column
x = []
for i in range(len(df)):
    if df['Time_Order_picked_min'][i] < df['Time_Orderd_min'][i]:
        x.append(df['Time_Order_picked_min'][i] + (60-df['Time_Orderd_min'][i]))
    else:
        x.append(df['Time_Order_picked_min'][i] - df['Time_Orderd_min'][i])
df['Time_diff_bw_order_and_pickup'] = x
df.drop(columns=['Time_Orderd_min','Time_Order_picked_min'],axis=1,inplace=True)

In [161]:
#Segregate numerical columns and categorical columns
numerical_columns = df.columns[df.dtypes!='object']
categorical_columns = df.columns[df.dtypes=='object']
print('numerical_columns',numerical_columns)
print('categorical_columns',categorical_columns)

numerical_columns Index(['Delivery_person_Age', 'Delivery_person_Ratings',
       'Delivery_location_latitude', 'Delivery_location_longitude',
       'Vehicle_condition', 'multiple_deliveries', 'Time_taken (min)',
       'Time_diff_bw_order_and_pickup'],
      dtype='object')
categorical_columns Index(['Delivery_person_ID', 'Weather_conditions', 'Road_traffic_density',
       'Type_of_order', 'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')


In [162]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [163]:
categories = ['Fog','Stormy','Sandstorms','Windy','Cloudy','Sunny'],['Jam', 'High', 'Medium', 'Low'],['Drinks','Snack','Meal','Buffet'],['bicycle','electric_scooter','scooter','motorcycle'],['Yes','No'],['Semi-Urban','Urban','Metropolitian']
ordinal_cols = df[categorical_columns].drop(columns='Delivery_person_ID',axis=1)

In [ ]:
whether = []

In [171]:
#Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ]
)

#Categorical pipeline
cat_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OrdinalEncoder(categories=categories)),
    ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('numerical pipeline', num_pipeline,numerical_columns),
    ('categorical pipeline',cat_pipeline,categorical_columns)
])

In [52]:
# from sklearn.impute import SimpleImputer
# num_imputer = SimpleImputer(strategy='median')
# num_imputer.fit(df[numerical_columns])
# df[numerical_columns] = num_imputer.transform(df[numerical_columns])

In [53]:
# cat_imputer = SimpleImputer(strategy='most_frequent')
# df[categorical_columns] = cat_imputer.fit_transform(df[categorical_columns])

In [54]:
# df['Delivery_person_Age'] = df['Delivery_person_Age'].astype(int)
# df['multiple_deliveries'] = df['multiple_deliveries'].astype(int)